<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install gdown
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

In [12]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [13]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

resnet50_50-epochs_stl10 1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu


In [14]:
# download and extract model files
os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
os.system('unzip {}'.format(folder_name))
!ls

checkpoint_0040.pth.tar
config.yml
events.out.tfevents.1610927742.4cb2c837708d.2694093.0
resnet50_50-epochs_stl10.zip
sample_data
training.log


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('../datasets', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('../datasets', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('../datasets', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('../datasets', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [29]:
with open(os.path.join('./config.yaml')) as file:
  config = yaml.load(file, Loader=yaml.FullLoader)
# config['arch'] = 'resnet18'
config['arch']

'resnet18'

In [32]:
if config['arch'] == 'resnet18':
  model = torchvision.models.resnet18(weights=None, num_classes=10).to(device)
elif config['arch'] == 'resnet50':
  model = torchvision.models.resnet50(weights=None, num_classes=10).to(device)

In [31]:
ckpt_dir = r'..\runs\Mar29_17-30-54_resnet18'
checkpoint = torch.load(f'{ckpt_dir}/checkpoint_0100.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [33]:
log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [11]:
if config['dataset_name'] == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config['dataset_name'] == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)
print("Dataset:", config['dataset_name'])

Files already downloaded and verified
Files already downloaded and verified
Dataset: stl10


In [34]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [14]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [36]:
epochs = 100
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 19.72426414489746	Top1 Test accuracy: 32.52197265625	Top5 test acc: 82.158203125
Epoch 1	Top1 Train accuracy 41.60271072387695	Top1 Test accuracy: 47.24365234375	Top5 test acc: 93.0029296875
Epoch 2	Top1 Train accuracy 51.19140625	Top1 Test accuracy: 53.67431640625	Top5 test acc: 95.44921875
Epoch 3	Top1 Train accuracy 56.06043243408203	Top1 Test accuracy: 56.90673828125	Top5 test acc: 96.1572265625
Epoch 4	Top1 Train accuracy 59.019989013671875	Top1 Test accuracy: 58.8916015625	Top5 test acc: 96.52099609375
Epoch 5	Top1 Train accuracy 60.6755485534668	Top1 Test accuracy: 60.107421875	Top5 test acc: 96.865234375
Epoch 6	Top1 Train accuracy 61.93819046020508	Top1 Test accuracy: 60.96923828125	Top5 test acc: 97.01904296875
Epoch 7	Top1 Train accuracy 62.77803421020508	Top1 Test accuracy: 61.6357421875	Top5 test acc: 97.26318359375
Epoch 8	Top1 Train accuracy 63.40303421020508	Top1 Test accuracy: 62.20703125	Top5 test acc: 97.3974609375
Epoch 9	Top1 Train accur

In [37]:
top1_accuracy = []
top5_accuracy = []
for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy.append(top1[0].cpu().numpy())
    top5_accuracy.append(top5[0].cpu().numpy())

print("top1 accuracy:", np.mean(top1_accuracy), np.std(top1_accuracy))
print("top5 accuracy:", np.mean(top5_accuracy), np.std(top5_accuracy))


top1 accuracy: 70.52734 2.319002
top5 accuracy: 98.38379 0.5431786


In [27]:
ckpt_name = 'resnet18_stl10-stl10' # resnet18_stl10-stl10
torch.save(model.state_dict(), f'weights/{ckpt_name}.pth')
